In [526]:
import configparser
import os

from utils import util_data, util_preprocess


In [4]:
config = configparser.ConfigParser()
config.read("config.ini")


['config.ini']

In [530]:
ds_conf = config["GSMARENA_DS"]
# Download the training and test datasets
dh = util_data.DownloadHelper(
    ds_conf["url"], ds_conf["name"], ds_conf["mode"], quiet=False
)

dh.download()

File gsm.csv already exists. Skip download.


In [527]:
df = dh.read_csv(low_memory=False)
df = (
    df.pipe(util_preprocess.filter_unwanted_columns)
    .pipe(util_preprocess.preprocess_feature, "misc_price")
    .pipe(util_preprocess.preprocess_feature, "launch_announced")
    .pipe(util_preprocess.preprocess_feature, "display_size")
    .pipe(util_preprocess.preprocess_feature, "display_type")
    .pipe(util_preprocess.preprocess_feature, "display_resolution")
    .pipe(util_preprocess.preprocess_feature, "memory_card_slot")
    .pipe(util_preprocess.preprocess_feature, "sound_loudspeaker")
    .pipe(util_preprocess.preprocess_feature, "sound_3.5mm_jack")
    .pipe(util_preprocess.preprocess_feature, "comms_wlan")
    .pipe(util_preprocess.preprocess_feature, "comms_nfc")
    .pipe(util_preprocess.preprocess_feature, "memory_internal")
    .pipe(util_preprocess.preprocess_feature, "network_technology")
    .pipe(util_preprocess.preprocess_feature, "battery")
    .pipe(util_preprocess.preprocess_feature, "battery_charging")
    .pipe(util_preprocess.preprocess_feature, "body")
    .pipe(util_preprocess.preprocess_feature, "main_camera_cols")
    .pipe(util_preprocess.preprocess_feature, "selfie_camera_cols")    
)


Removing unwanted columns and duplicates...
Shape before removing unwanted columns and duplicates: (10679, 86)
Shape after removing unwanted columns and duplicates: (9919, 24)

Number of rows before extracting valid misc_price: 9919
Number of rows after extracting valid misc_price: 6151

Number of rows before extracting valid launch_announced: 6151
Number of rows after extracting valid launch_announced: 6142

Number of rows before extracting valid display_size: 6142
Number of rows after extracting valid display_size: 6054

Number of rows before extracting valid display_type: 6054
Number of rows after extracting valid has_oled_display: 6054

Number of rows before extracting valid display_resolution: 6054
Number of rows after extracting valid ['display_width', 'display_height']: 6027

Number of rows before extracting valid memory_card_slot: 6027
Number of rows after extracting valid has_memory_card_slot: 6027

Number of rows before extracting valid sound_loudspeaker: 6027
Number of rows 

In [523]:
print(f"The dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

The dataframe has 2647 rows and 21 columns.


Filter conditions for recent mobile phones

- launch_announced >= 2014
- display_size in (4, 7.5)
- display_height and display_width > 512

In [521]:
filter_condition = {
    "launch_announced": lambda x: x >= 2014,
    "display_size": lambda x: x >= 4,
    "display_size": lambda x: x <= 7.5,
    "display_height": lambda x: x >= 512,
    "display_width": lambda x: x >= 512,
}
df = util_preprocess.filter_unwanted_rows(df, filter_condition)

Removing unwanted rows and duplicates...
Shape before removing unwanted rows and duplicates: (2647, 21)
Shape after removing unwanted rows and duplicates: (1973, 21)



In [532]:
out_path = os.path.join("data", "processed", ds_conf["name"])
df.to_csv(out_path, index=False)